In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [9]:
## Section 2: Dataset Preparation
data_dir = 'Images'  # Ensure this directory exists and contains subdirectories for each class

# Validate Dataset
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Dataset directory {data_dir} not found.")

# categories = os.listdir(data_dir)
# categories = [cat for cat in categories if os.path.isdir(os.path.join(data_dir, cat))]
# num_classes = len(categories)

print(f"Categories Found: {categories}")
print(f"Number of Classes: {num_classes}")

if num_classes == 0:
    raise ValueError("No categories found in the dataset directory.")

Categories Found: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
Number of Classes: 5


In [3]:
## Section 3: ImageDataGenerator
img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

# Debugging Data Generator
for data_batch, labels_batch in train_generator:
    print(f"Image batch shape: {data_batch.shape}")
    print(f"Labels batch shape: {labels_batch.shape}")
    break

Found 2198 images belonging to 5 classes.
Found 548 images belonging to 5 classes.
Image batch shape: (32, 224, 224, 3)
Labels batch shape: (32, 5)


In [5]:
## Section 4: Model Definition
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')  # Ensure num_classes matches categories
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [ ]:
## Section 5: Model Training
epochs = 1
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss:.2f}, Validation Accuracy: {val_acc:.2f}")

model.save('FLower_model.h5')
print("Model saved as 'FLower_model.h5'")

c:\Users\PMYLS\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


69/69 ━━━━━━━━━━━━━━━━━━━━ 330s 5s/step - accuracy: 0.4503 - loss: 3.4047 - val_accuracy: 0.7719 - val_loss: 0.6465
18/18 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.7957 - loss: 0.6100


Validation Loss: 0.65, Validation Accuracy: 0.77
Model saved as 'animal10_model.h5'


In [ ]:
## Section 6: Model Testing and Prediction
try:
    # Load the model in the new Keras format
    model = load_model('FLower_model.keras')
    print("Model loaded successfully.")
    model.summary()

    # Create a dummy input for testing
    example_input = np.random.random((1, 224, 224, 3)).astype(np.float32)

    # Make predictions
    prediction = model.predict(example_input)
    predicted_class = categories[np.argmax(prediction)]
    print(f"Predicted Class: {predicted_class}")

    # Display dummy image with prediction
    plt.imshow(example_input[0])
    plt.title(f"Predicted Class: {predicted_class}")
    plt.show()

except Exception as e:
    print(f"Error during model testing: {e}")


Error during model testing: File not found: filepath=animal10_model.keras. Please ensure the file is an accessible `.keras` zip file.


In [ ]:
## Section 7: Fine-tuning the Model (Optional)
train_data = np.random.random((100, 224, 224, 3))  # Replace with actual training data
train_labels = np.random.randint(0, len(categories), (100,))  # Replace with actual labels

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=5)
model.save('updated_FLower_model.h5')
print("Updated model saved as 'updated_FLower_model.h5'")

Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.1508 - loss: 2.2987
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.2130 - loss: 2.5710
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.2325 - loss: 2.2859
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.1955 - loss: 2.0062
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.2033 - loss: 1.8075


Updated model saved as 'updated_animal10_model.h5'
